In [ ]:
from src.io_utils import load_data, coerce_numeric

df = coerce_numeric(load_data("data/library_loans.csv"))
df = df.dropna(subset=["loan_days"]).copy()
df = df[df["loan_days"] > 0].copy()

df["month"] = df["checkout_date"].dt.to_period("M").dt.to_timestamp()
df["week"] = df["checkout_date"].dt.to_period("W").dt.start_time

df.info()
# df.head(3)


In [ ]:
from src import metrics as M

print("Totalt antal lån:", M.total_loans(df))
print("Genomsnittlig lånetid (dagar:)", f"{M.avg_loan_days(df):.1f}")
print("Andel övertid (%)", f"{100 * M.overdue_rate(df):.1f}%")

In [ ]:
by_genre = M.loans_by_genre(df)
by_branch = M.loans_by_branch(df)
by_genre
by_branch

In [ ]:
ts_month = M.loans_over_time(df, freq="M")
ts_month.head()

In [ ]:
import matplotlib.pyplot as plt
from src import viz as V

# Lån per genre
fig, ax = plt.subplots(figsize=(9, 4))
V.bar(ax, by_genre["genre"], by_genre["loans"], "Lån per genre", "Genre", "Antal lån")

# Top-10 filialer
fig, ax = plt.subplots(figsize=(9, 4))
top_10 = by_branch.head(10)
V.bar(ax, top_10["branch"], top_10["loans"], "Topp 10 filialer", "Filial", "Antal lån")

# Lån per månad
fig, ax = plt.subplots(figsize=(9, 4))
V.line(ax, ts_month["checkout_date"], ts_month["loans"], "Lån per månad", "Månad", "Antal lån")